### Rating Sports Teams with Current ACC Basketball Data

This notebook will run through a process of rating and ranking ACC basketball teams using the current game data from 03/01/2020.

The block below imports data from https://www.masseyratings.com/scores.php?s=309912&sub=10423&all=1&mode=3&format=1.

The columns are team 1, a 1 for a win or -1 for a loss, the points team 1 scored, and then the next three columns are the same but for team 2.

In [ ]:
import pandas
import numpy as np

df = pandas.read_csv('03_01_2020_ACCgames.csv')
print(df)

The block below creates the X matrix to hold the matchups and the p vector to hold the point differentials.

In [ ]:
games = 135
teams = 15
X = np.zeros((games, teams))
p = np.zeros((teams,1))

The block below iterates through the data frame of game data to populate the X matrix with matchup data and the p vector with point differentials.

In [ ]:
for k in range (0, games):
    col = df['team1'][k] - 1
    col2 = df['team2'][k] - 1
    
    for1 = df['points1'][k]-df['points2'][k]
    for2 = df['points2'][k]-df['points1'][k]
    
    p[col] = p[col]+for1
    p[col2]= p[col2]+for2
    
    if (for1 > for2):
        wL = 1
        wL2 = -1
    else:
        wL = -1
        wL2 = 1
        
    X[k][col] = wL
    X[k][col2] = wL2

We can see some of our X matrix here:

In [ ]:
print(X)

Now we will create the M matrix and verify that it looks correct, with the bottom row changed to all ones.

In [ ]:
M = (X.transpose())@X
M[14,:] = np.ones(teams)

print(M)

We also have to change the final entry of our p vector to match the change made in M.

In [ ]:
p[14] = 0
print(p)

We can now calculate our ratings for the teams with the Mr = p equation.

In [ ]:
r = np.linalg.solve(M, p)

print(r)

Now we may want to visualize these ratings in a nicer way.

In [ ]:
ratings_df = pandas.DataFrame(r, columns=['Rating'],
                index = ['Boston_College', 'Clemson', 'Duke',
                         'Florida_St', 'Georgia_Tech', 'Louisville', 
                         'Miami_FL', 'NC_State', 'North_Carolina', 
                         'Notre_Dame', 'Pittsburgh', 'Syracuse',
                         'Virginia', 'Virginia_Tech', 'Wake_Forest'])

ratings_df.sort_values('Rating', inplace=True, ascending=False)
ranking = [x+1 for x in range(teams)]

ratings_df['Ranking'] = ranking

ratings_df

Now let's take the next steps of adding offensive and defensive ratings.

We start by creating the diagonal T matrix, and then subtract to obtain the off-diagonal P matrix.  The new M1 matrix is created because we no longer alter the M matrix the way we did earlier.

In [ ]:
M1 = (X.transpose())@X
T = (np.diag(np.diag(M1)))
print(T)

In [ ]:
P = T - M1
print(P)

We can now calculate our f, a, and p vectors.  The new p1 vector is created because we no longer alter the p vector the way we did earlier.

In [ ]:
f = np.zeros((teams,1))
a = np.zeros((teams,1))
p1 = np.zeros((teams,1))

for k in range (0, games):
    col = df['team1'][k] - 1
    col2 = df['team2'][k] - 1
    
    for1 = df['points1'][k]-df['points2'][k]
    for2 = df['points2'][k]-df['points1'][k]
    
    p1[col] = p1[col]+for1
    p1[col2]= p1[col2]+for2
    
    all_for1 = df['points1'][k]
    all_against1 = df['points2'][k]
    all_for2 = df['points2'][k]
    all_against2 = df['points1'][k]
    
    f[col] = f[col]+all_for1
    f[col2]= f[col2]+all_for2
    
    a[col] = a[col]+all_against1
    a[col2] = a[col2]+all_against2

We can verify that our f, a, and p vectors look appropriate.

In [ ]:
print(f)

In [ ]:
print(a)

This verifies that our p1 vector is indeed equal to f - a.

In [ ]:
print(f - a)
print(p1)

We now calculate the defensive rating, d, using the system of equations that we derived previously.

In [ ]:
d = np.linalg.solve((T+P),((T@r)-f))
print(d)

Using our existing d and r vectors, we can find the offensive rating, o.

In [ ]:
o = r - d
print(o)

Once again we will view our ratings in a prettier way.

In [ ]:
def_ratings_df = pandas.DataFrame(d, columns=['Defensive Rating'],
                index = ['Boston_College', 'Clemson', 'Duke',
                         'Florida_St', 'Georgia_Tech', 'Louisville', 
                         'Miami_FL', 'NC_State', 'North_Carolina', 
                         'Notre_Dame', 'Pittsburgh', 'Syracuse',
                         'Virginia', 'Virginia_Tech', 'Wake_Forest'])

def_ratings_df.sort_values('Defensive Rating', inplace=True, ascending=False)
ranking_d = [x+1 for x in range(teams)]

def_ratings_df['Defensive Ranking'] = ranking_d

def_ratings_df

In [ ]:
off_ratings_df = pandas.DataFrame(o, columns=['Offensive Rating'],
                index = ['Boston_College', 'Clemson', 'Duke',
                         'Florida_St', 'Georgia_Tech', 'Louisville', 
                         'Miami_FL', 'NC_State', 'North_Carolina', 
                         'Notre_Dame', 'Pittsburgh', 'Syracuse',
                         'Virginia', 'Virginia_Tech', 'Wake_Forest'])

off_ratings_df.sort_values('Offensive Rating', inplace=True, ascending=False)
ranking_o = [x+1 for x in range(teams)]

off_ratings_df['Offensive Ranking'] = ranking_o

off_ratings_df